# Research into Chatbot stuff + Langchain

In [21]:
import os, asyncio, datetime as dt
from dotenv import load_dotenv
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import shutil

import textwrap

from dotenv import load_dotenv
load_dotenv("../.env.local")

# OLLAMA_MODEL = "gpt-oss-20b"
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL")
OLLAMA_PORT = os.getenv("OLLAMA_PORT")
OLLAMA_HOST = os.getenv("OLLAMA_HOST")

BASE_URL = f"http://{OLLAMA_HOST}:{OLLAMA_PORT}"

print(f"Using Ollama model: {OLLAMA_MODEL} at {OLLAMA_HOST}:{OLLAMA_PORT}")

Using Ollama model: gpt-oss:20b at localhost:11434


In [ ]:
# 1) LLM - Configure to include raw response with thinking
llm = ChatOllama(
    model=OLLAMA_MODEL, 
    base_url=BASE_URL,
    temperature=0.7,
    # Try to enable thinking/reasoning mode
    # Some models need specific system prompts or parameters
)

## Model Information

**Important:** To see `<thinking>` tags, you need a model that supports reasoning tokens, such as:
- `deepseek-r1` or `deepseek-r1:latest`
- `qwen2.5` with thinking enabled
- Other reasoning-capable models

The `gpt-oss:20b` model may not expose thinking tokens by default.

In [40]:
# 2) Prompt with history
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are concise and helpful."),
        MessagesPlaceholder(variable_name="history"),
        ("user", "{input}"),
    ]
)

In [50]:
# 3) Memory store (per session_id)
_store: dict[str, InMemoryChatMessageHistory] = {}


def get_history(session_id: str) -> InMemoryChatMessageHistory:
    return _store.setdefault(session_id, InMemoryChatMessageHistory())

In [42]:
# 4) Chain with message history
chain = RunnableWithMessageHistory(
    prompt | llm,
    get_session_history=get_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [43]:
def ask(session_id: str, text: str) -> str:
    resp = chain.invoke(
        {"input": text},
        config={"configurable": {"session_id": session_id}},
    )
    return resp.content


def reset(session_id: str):
    _store.pop(session_id, None)

In [51]:
sid = "peter"
print(ask(sid, "Hello, who are you?"))
print(ask(sid, "Remember my name and greet me briefly. It's Peter"))
print(ask(sid, "What did I tell you earlier?"))

I’m ChatGPT, a large language model created by OpenAI. I can help answer questions, explain topics, or assist with many tasks you have.
Hello, Peter! 👋 How can I help you today?
Hello, Peter! 👋 How can I help you today?
You told me that your name is Peter.
You told me that your name is Peter.


In [46]:
print(_store)

{'test_thinking': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is 12 * 8?', additional_kwargs={}, response_metadata={}), AIMessage(content='96', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-11-10T02:13:07.7038998Z', 'done': True, 'done_reason': 'stop', 'total_duration': 881251600, 'load_duration': 141882200, 'prompt_eval_count': 89, 'prompt_eval_duration': 119679700, 'eval_count': 48, 'eval_duration': 602206700, 'model_name': 'gpt-oss:20b', 'model_provider': 'ollama'}, id='lc_run--65ab2876-1c36-4876-b356-a209fba38bd9-0', usage_metadata={'input_tokens': 89, 'output_tokens': 48, 'total_tokens': 137})]), 'peter': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello, who are you?', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi! I’m ChatGPT, an AI language model created by OpenAI. I’m here to answer questions, help with tasks, and chat about a wide range of topics. How can I assist you today?', addit

# Debugging

In [52]:
import re
import json

TERM_W = shutil.get_terminal_size((100, 20)).columns
WRAP_W = max(60, min(120, TERM_W - 4))

ROLE_TAGS = {
    "system": "SYSTEM",
    "user": "USER",
    "assistant": "ASSISTANT",
    "tool": "TOOL",
}


def _now():
    return dt.datetime.now().strftime("%H:%M:%S")


def format_block(role: str, content: str, message=None) -> str:
    role = ROLE_TAGS.get(role.lower(), role.upper())
    
    # Extract thinking section if present
    thinking_match = re.search(r'<thinking>(.*?)</thinking>', content, re.DOTALL)
    thinking_text = ""
    display_content = content
    
    if thinking_match:
        thinking_text = thinking_match.group(1).strip()
        # Remove thinking tags from main content
        display_content = re.sub(r'<thinking>.*?</thinking>', '', content, flags=re.DOTALL).strip()
    
    result = []
    border = "─" * min(WRAP_W, 80)
    
    # Add thinking section if present
    if thinking_text:
        thinking_wrapped = textwrap.fill(thinking_text, width=WRAP_W)
        result.append(f"[{_now()}] {role} (THINKING)")
        result.append(thinking_wrapped)
        result.append(border)
    
    # Check for tool calls in the message
    if message and hasattr(message, 'tool_calls') and message.tool_calls:
        result.append(f"[{_now()}] {role} (TOOL CALLS)")
        for tool_call in message.tool_calls:
            tool_info = f"Tool: {tool_call.get('name', 'unknown')}"
            if 'args' in tool_call:
                tool_info += f"\nArgs: {json.dumps(tool_call['args'], indent=2)}"
            if 'id' in tool_call:
                tool_info += f"\nID: {tool_call['id']}"
            result.append(textwrap.fill(tool_info, width=WRAP_W))
        result.append(border)
    
    # Check for additional_kwargs that might contain tool info
    if message and hasattr(message, 'additional_kwargs') and message.additional_kwargs:
        if 'tool_calls' in message.additional_kwargs:
            result.append(f"[{_now()}] {role} (TOOL CALLS - RAW)")
            result.append(textwrap.fill(str(message.additional_kwargs['tool_calls']), width=WRAP_W))
            result.append(border)
    
    # Add main content
    if display_content:
        wrapped = textwrap.fill(display_content, width=WRAP_W)
        result.append(f"[{_now()}] {role}")
        result.append(wrapped)
        result.append(border)
    
    return "\n".join(result)


def print_history(session_id: str):
    hist = get_history(session_id)
    print("\n" + "=" * 12 + f" Conversation: {session_id} " + "=" * 12)
    for m in hist.messages:
        # m is HumanMessage/AIMessage/SystemMessage/ToolMessage
        print(format_block(m.type, m.content, message=m))
    print("=" * (26 + len(session_id)))


print_history(sid)


============ Conversation: peter ============
[21:14:53] HUMAN
Hello, who are you?
────────────────────────────────────────────────────────────────────────────────
[21:14:53] AI
I’m ChatGPT, a large language model created by OpenAI. I can help answer questions, explain
topics, or assist with many tasks you have.
────────────────────────────────────────────────────────────────────────────────
[21:14:53] HUMAN
Remember my name and greet me briefly. It's Peter
────────────────────────────────────────────────────────────────────────────────
[21:14:53] AI
Hello, Peter! 👋 How can I help you today?
────────────────────────────────────────────────────────────────────────────────
[21:14:53] HUMAN
What did I tell you earlier?
────────────────────────────────────────────────────────────────────────────────
[21:14:53] AI
You told me that your name is Peter.
────────────────────────────────────────────────────────────────────────────────
